In [ ]:
import numpy as np
import pandas as pd
import logging
import pickle
import os 
import matplotlib.pyplot as plt

datadir = "5k_data_from_yuxuan/"

# 5000 patterns. Each pattern consists of indices characterized by which chnanels are modulated oen of 4 delay modes. 44 stimulating channels are used (though some pattern indices have no stimulation on any channels). 
# Each stim is 3 uA amplitude, biphasic, 167-66-167 us biphasic pulses with 66 us interphase interval. 
# approx 30kHz sampling rate but imprecise.

# Each stimulation is delivered in 600 ms period, with 1400 inter-stimulation interval. Thus, 0.2 s per stimulation * 5000 stimulations = 1000 seconds total experiment.

pattern_registrations = pickle.load(open(os.path.join(datadir, "pattern_registrations.pkl"), "rb"))

# spkVecs contains spike times for all neurons. 47 are recorded.
spikes_df = pd.DataFrame(np.load(os.path.join(datadir, "spkVecs.npy"))) # size = (3396476,))
spikes_df.columns = ['timestamp', 'neuron_id', 'segment_index']
spikes_df.drop(columns=['segment_index'], inplace=True)
spikes_df['timestamp'] = spikes_df['timestamp'].astype(int)
spikes_df['neuron_id'] = spikes_df['neuron_id'].astype(int) # noncontinuous neuron ids from with numbers corresponding to shank location
spikes_df.sort_values(by=['timestamp', 'neuron_id'], inplace=True)
print (spikes_df.shape)
print (spikes_df['neuron_id'].nunique(), " unique neurons recorded")
print (spikes_df['timestamp'].max()/1000, " seconds of recording")
spikes_df.head(5)

(3396476, 2)
47  unique neurons recorded
328503.754  seconds of recording


,timestamp,neuron_id
0,122,206
1,196,5
2,244,200
3,246,300
4,340,200


In [ ]:
import pandas as pd
import numpy as np

def read_pattern_json(data):
    # 1. Flatten the Pattern -> Steps level
    # We use record_path to reach the 'steps' and meta to keep parent info
    df = pd.json_normalize(
        data, 
        record_path=['steps'], 
        meta=[
            'pattern_name', # given name of the pattern
            'pattern_lambda', # given lambda parameter that generated the pattern 
            'pattern_flag_start_timestamp', # given starting timpestamp of pattern
            'pattern_timing_index' # given order of pattern from 1-5,000 (first to last)
        ],
        record_prefix='step_'
    )
    # 2. Flatten the 'step_channel_delays' list into individual rows
    # This creates a row for every delay entry. Steps with [] will become NaN.
    df = df.explode('step_channel_delays').reset_index(drop=True)

    # 3. Convert the dictionaries in 'step_channel_delays' into separate columns
    delays_df = pd.json_normalize(df['step_channel_delays'])
    
    # 4. Store pattern length
    df['pattern_idx_length'] = df.groupby('pattern_timing_index')['step_index'].transform('max') + 1

    # 6. Combine and cleanup
    final_df = pd.concat([df.drop(columns=['step_channel_delays']), delays_df], axis=1)
    
    return final_df

# Usage:
pattern_df = read_pattern_json(pattern_registrations)
pattern_df.head(30)
max_pattern_timestamp = pattern_df['step_start_timestamp'].max()
min_pattern_timestamp = pattern_df['step_start_timestamp'].min()
print (f"Min pattern timestamp: {min_pattern_timestamp}")
print (f"Max pattern timestamp: {max_pattern_timestamp}")



Max pattern timestamp: 328442242
Max pattern timestamp: 905905


In [ ]:
pattern_df['pattern_idx_length'].value_counts()

# how many times each pattern appears in separate timestamps
pattern_counts = pattern_df[['pattern_name', 'pattern_timing_index']].drop_duplicates().groupby('pattern_name').size()



,end_time
count,5.000000e+03
mean,1.608243e+08
std,9.449670e+07
min,9.232250e+05
25%,7.871570e+07
50%,1.589370e+08
75%,2.417233e+08
max,3.284422e+08


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

def plot_spike_raster(df, neurons=None, time_range=None, tick_height=0.8):
    """
    Plots a spike raster plot from a DataFrame.
    
    Parameters:
    - df: DataFrame with 'timestamp' and 'neuron_id'
    - neurons: List of neuron IDs to plot (None = all)
    - time_range: Tuple of (min_time, max_time) to zoom in
    - tick_height: Vertical scale of the spike marks
    """
    
    # 1. Filter by Time Interval (if provided)
    if time_range:
        df = df[(df['timestamp'] >= time_range[0]) & (df['timestamp'] <= time_range[1])]
    
    # 2. Filter by Neuron IDs (if provided)
    if neurons is not None:
        df = df[df['neuron_id'].isin(neurons)]
    
    # 3. Group timestamps by neuron_id
    # We create a list of arrays, where each array contains timestamps for one neuron
    grouped = df.groupby('neuron_id')['timestamp'].apply(np.array)
    
    # Prepare data for eventplot
    spike_data = grouped.values
    neuron_labels = grouped.index.values

    # 4. Plotting
    plt.figure(figsize=(12, 6))
    plt.eventplot(spike_data, lineoffsets=neuron_labels, linelengths=tick_height, color='black')

    plt.title('Spike Raster Plot')
    plt.xlabel('Time (s)')
    plt.ylabel('Neuron ID')
    
    # Set y-ticks to show specific neuron IDs clearly
    plt.yticks(neuron_labels)
    plt.grid(axis='x', linestyle='--', alpha=0.5)
    
    plt.tight_layout()
    plt.show()

# Example Usage:
# Plot all neurons
# plot_spike_raster(spkVecs)

import matplotlib.pyplot as plt
import os

def plot_per_stimulation_rasters(spkVecs, pattern_df, output_dir='rasters'):
    """
    Creates a separate raster plot for every stimulation event in pattern_df.
    
    Parameters:
    - spkVecs: DataFrame with spike ['timestamp', 'neuron_id']
    - pattern_df: DataFrame with ['step_index', 'step_start_timestamp', 'pattern_name']
    - output_dir: Directory to save the figures
    """
    # Create output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # 1. Identify unique stimulation steps
    # We group by step_index to get the start time of each pattern
    patterns = pattern_df.groupby(['pattern_timing_index', 'pattern_name'])['pattern_flag_start_timestamp'].first().reset_index()
    
    # 2. Determine the end time for each step (start of the next step)
    patterns['pattern_end_timestamp'] = patterns['pattern_flag_start_timestamp'].shift(-1)
    
    # Handle the very last pattern (which has no 'next' start time)
    # We can add a fixed duration or use the last spike time
    last_spike_time = spkVecs['timestamp'].max()
    patterns['pattern_end_timestamp'] = patterns['pattern_end_timestamp'].fillna(last_spike_time)
    y_positions = np.arange(len(spkVecs['neuron_id'].unique()))
    y_labels = spkVecs['neuron_id'].unique()
    # 3. Iterate through each stimulation and plot
    for i, row in patterns.iterrows():
        idx = int(row['pattern_timing_index'])
        t_start = row['pattern_flag_start_timestamp']
        t_end = row['pattern_end_timestamp']
        name = row['pattern_name']
        
        # Filter spikes for this interval
        mask = (spkVecs['timestamp'] >= t_start) & (spkVecs['timestamp'] < t_end)
        interval_spikes = spkVecs[mask]
        
        if interval_spikes.empty:
            print(f"Skipping Pattern ({name}): No spikes found between {t_start} and {t_end}")
            continue

        # Group data for eventplot
        grouped = interval_spikes.groupby('neuron_id')['timestamp'].apply(np.array)
        
        # Plotting
        plt.figure(figsize=(12, 6))
        # Use the marker '|' to simulate the vertical tick look
        plt.scatter(pattern_df['timestamp'], pattern_df['neuron_id'], 
                    marker='|', color='black', s=100, linewidths=0.5)
        
        plt.title(f"Pattern {name} | Time: {t_start/1000} to {t_end/1000} s")
        plt.xlabel('Time (ms)')
        plt.ylabel('Neuron ID')
        plt.yticks(y_positions, y_labels)
        plt.xlim(t_start, t_end) # Ensure the x-axis matches the pattern window
        plt.grid(axis='x', linestyle='--', alpha=0.5)
        
        # Save figure
        file_path = os.path.join(output_dir, f"raster_pattern_{name}.png")
        plt.savefig(file_path)
        plt.close() # Close to free up memory
        
    print(f"Finished! {len(steps)} potential plots processed and saved to '{output_dir}'.")

# Run the function
plot_per_stimulation_rasters(spkVecs, pattern_df, "figures/patterns/rasters")